In [1]:
import os
import numpy as np
import pandas as pd
import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from ipywidgets import widgets

In [2]:
file_name='eda_final.csv'
file_path = os.path.join(os.path.abspath(""), "PYTHON", file_name)
df = pd.read_csv(file_name,delimiter=',')

In [3]:
df=df.rename(columns={"ap_hi": "systolic", "ap_lo": "diastolic"})
df['gender'] = df['gender'].replace(1,'Female')
df['gender'] = df['gender'].replace(2,'Male')
df['age'] = df['age'].apply(np.floor)
df[['age','gender','systolic', 'diastolic']]
age_min = df['age'].min()
age_max = df['age'].max()

In [4]:
age_from = widgets.IntSlider(
    value=age_min,
    min=age_min,
    max=age_max,
    step=1.0,
    description='Age from:',
    continuous_update=False
)

In [5]:
age_to = widgets.IntSlider(
    value=age_max,
    min=age_min,
    max=age_max,
    step=1.0,
    description='Age to:',
    continuous_update=False
)

In [6]:
gender = widgets.Dropdown(
    options=list(df['gender'].unique()),
    value='Male',
    description='Gender:',
)

In [7]:
use_gender = widgets.Checkbox(
    value=False,
    description='Gender',
)

In [8]:
systolic = go.Histogram(x=df['systolic'], opacity=0.75, name='Systolic')
diastolic = go.Histogram(x=df['diastolic'], opacity=0.75, name='Diastolic')
g = go.FigureWidget(data=[systolic,diastolic],
                    layout=go.Layout(
                        title=dict(
                            text='BLOOD PRESSURE GRAPH'
                        ),
                        barmode='overlay'
                    ))

In [9]:
 def response(change):
    if use_gender.value:
                filter_list = [i and j and k for i, j, k in
                            zip(df['gender'] == gender.value, df['age'] >= age_from.value,
                                df['age'] <= age_to.value)]
                temp_df = df[filter_list]
    else:
        filter_list = [i and j for i, j in
                    zip(df['age'] >= age_from.value, df['age'] <= age_to.value)]
        temp_df = df[filter_list]
    x1 = temp_df['systolic']
    x2 = temp_df['diastolic']
    with g.batch_update():
        g.data[0].x = x1
        g.data[1].x = x2
        g.layout.barmode = 'overlay'
        g.layout.xaxis.title = 'Person'
        g.layout.yaxis.title = 'Pressure'

age_to.observe(response, names="value")
age_from.observe(response, names="value")
gender.observe(response, names="value")
use_gender.observe(response, names="value")

In [10]:
container = widgets.HBox(children=[age_from, age_to])
container2 = widgets.HBox(children=[use_gender, gender])

In [11]:
widgets.VBox([container2, container, g])